### Import library

In [ ]:
import numpy as np
import pandas as pd
import warnings
import os
import random
import pickle
# import ML related libraries
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import SGD, Adam
from keras.models import Model
from keras.layers import Dense, Input, Dropout
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from bayes_opt import BayesianOptimization
# import plotting related libraries
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from scipy import interpolate

# Things that we can change:)

In [ ]:
datapathway=r"C:\Users\wingt\Downloads\S11_along_BD_205.csv"
xstart=3            #column of xstart
xstop=5            #column of xend
ystart=6           #column of ystart
yname='Stress'      # name of the ultimate output array
yparameter='Position'       #position here but it will be what is the parameter to be tuned
xcolumn=['Travel_length', 'Welding_speed', 'Net_energy_input'] #name of inputs
act_list=[tf.nn.relu,tf.nn.tanh,tf.nn.sigmoid,tf.nn.elu,tf.nn.leaky_relu,tf.nn.softsign,tf.nn.softplus,tf.nn.softmax]
optimizer_search_list=['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
optimizer_tuning_number_of_full_update=200
optimizer_tuning_number_sample_used_in_one_training=10
grid_search_tuning_other_parameter_tuning_dict= {'N_hidden_nodes':[100, 500, 1000, 1500, 2000], 'l_rate':[0.0001, 0.001, 0.01], 'Batch_size':[1, 8, 16, 32], 'epochs':[1000]}
BO_turing_other_params_nn ={
     'N_hidden_nodes': (10, 2000),
     'l_rate':(0.0001, 0.01),
     'Batch_size':(1, 64),
     'epochs':(50, 1000),
 }
no_iter_before_conclude=5
no_iter_conclude=3

### Get Plot
It is a function plotting the actual simulation result and the prediction result

In [ ]:
def get_plot(oringin_data, predict_data, i):
    '''
    Plot target outputs against predicted outputs
    inputs:
        oringin_data: target outputs
        predict_data: predicted outputs
        i: plot index
        root: root for saving figure
    '''
    root = r'ANN_results/BD/test/test--' + str(i) + '.jpg'
    fig, ax = plt.subplots(figsize=(10,6))
    x = y_label_new
    oringin_data = oringin_data 
    predict_data = predict_data 
    p1 = ax.plot(x,oringin_data.ravel(),'r--', label = 'Target stress')
    p2 = ax.plot(x,predict_data.ravel(),'g--',label = 'Predict stress')
    ax.set_title("Test-Set" + str(i))
    # ax.set_xticks(x)
    ax.set_ylabel('Logitudinal stress (MPa)')
    ax.set_xlabel('Distance from top surface Z(mm)')
    
    ax.xaxis.set_major_locator(plt.MultipleLocator(1))
    ax.yaxis.set_tick_params(direction='out')

    ax2 = plt.twinx()
    difference = predict_data.ravel() - oringin_data.ravel()
    difference=np.array(list(map(lambda x,y:x/y,difference,oringin_data.ravel())))


    legend_elements = [Line2D([0], [0], color='red', lw=2, label='Simulation result'),
                       Line2D([0], [0], color='green', lw=2, label='ANN prediction')
                      ]

    ax.legend(handles=legend_elements, loc='best')
    plt.savefig(root)
    plt.close()

# Step 3: Tune the Optimizer
a function to tune the hyperparameter of the ANN machine-> optimizer
model used is from KerasRegressor
Used gridsearch cross validation
printing the best optimizer name and the score of this machine
printing the mean_test_score, std_test_score, parameter for all the optimizer tested

In [ ]:
def tune_optimizer():
    # callbacks = [
    #     EarlyStopping(monitor='mse', patience=100, verbose=0),
    # ]
    model = KerasRegressor(build_fn=create_model, nb_epoch=optimizer_tuning_number_of_full_update, batch_size=optimizer_tuning_number_sample_used_in_one_training, verbose=0) 
    optimizer = optimizer_search_list # 200 hidden nodes
    param_grid = dict(optimizer=optimizer) 
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=8)
    grid_result = grid.fit(Proc_X_train, Proc_Y_train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 
    for index, value in enumerate(grid_result.cv_results_['mean_test_score']):
        print("%f (%f) with: %r" % (grid_result.cv_results_['mean_test_score'][index], grid_result.cv_results_['std_test_score'][index], grid_result.cv_results_['params'][index]))

# Step 4: Tuning other parameters
a function tuning the other hyperparameter, including number of hidden nodes, learning rate, batch size,
set number of epochs to 1000
still use GridSearch CV and print the same thing as step 3

In [ ]:
def tune_other_para():
    model = KerasRegressor(build_fn=create_model_2, verbose=1) 
    param_grid = grid_search_tuning_other_parameter_tuning_dict
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=8)
    grid_result = grid.fit(Proc_X_train, Proc_Y_train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 
    for index, value in enumerate(grid_result.cv_results_['mean_test_score']):
        print("%f (%f) with: %r" % (grid_result.cv_results_['mean_test_score'][index], grid_result.cv_results_['std_test_score'][index], grid_result.cv_results_['params'][index]))

### Plot_history
plot learning curve
make learning history(generated later after building a model) a dataframe and insert a column called epoch in it
plot MSE against epoch
plot val_mse (should be validation mse) against no of epoch (validation error)

In [ ]:
def plot_history(history):
    '''
    Plot learning curve using NN training history info
    '''
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure()
    plt.xlabel('Epoch',fontsize=20)
    plt.ylabel('Mean Square Error [$MPG^2$]',fontsize=20)
    plt.plot(hist['epoch'], hist['mse'],
            label='Train Error')
    plt.plot(hist['epoch'], hist['val_mse'],
            label = 'Val Error')
    plt.ylim([0,0.4])
    plt.legend()
    plt.show()

### import data + preprocessing(select the suitable columns)
import the data using pd.read_csv
only get columns required
assign column names
get y data for training as well

In [ ]:
def data_import(csv_file_name):
    '''
    used to import dataset and split training and test dataset
    csv_file_name is the dateset root
    reture traing and test datasets
    '''
    raw_data = pd.read_csv(csv_file_name,header=None).dropna()
    x = raw_data.iloc[:,xstart:(xstop+1)]
    #x.columns = xcolumn
    y = raw_data.iloc[:,ystart:]
    return x, y

### Neural Network training function for Bayesian Optimisation
use a function for defning the machine name and parameters for the machine
use KerasRegressor to build the machine with specified hyperparameters
use processed training input and output (will define function to make them later) for training the model
history is the name of the model
return a scoe showing cross validation mse

In [ ]:
def nn_cl_bo(N_hidden_nodes, l_rate, Batch_size, epochs):
    '''
    Using certain number of neurons, learning rate and Batch size
    to train a NN and return its cross validation MSE
    inputs:
        N_hidden_nodes: number of neurons
        l_rate: learning rate
        Batch_size: Batch size
    output:
        score: cross validation MSE
    '''
    N_hidden_nodes = round(N_hidden_nodes)
    Batch_size = round(Batch_size)
    def nn_cl_fun():
        nn = keras.Sequential([
            keras.layers.Dense(N_hidden_nodes, activation=tf.nn.relu, input_shape=(4,)),
        # keras.layers.Dropout(0.5),
            keras.layers.Dense(1, activation='linear')
        ])
        nn.compile(loss='mse',
                    optimizer=tf.keras.optimizers.Adamax(learning_rate=l_rate, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.0),
                    metrics=['mse'])
        return nn
    nn = KerasRegressor(build_fn=nn_cl_fun, epochs=epochs, batch_size=Batch_size,
                        verbose=0)
    history = nn.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=1000, 
                        verbose=0, validation_split=0.1)
    score = -history.history['val_loss'][-1]
    return score

### function for creating a model for tuning optimiser

In [ ]:
def create_model(optimizer='adam'):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    model = keras.Sequential([
        keras.layers.Dense(200, activation=tf.nn.leaky_relu, input_shape=(4,)),
        keras.layers.Dense(21,activation='linear')
    ])
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse'])
    model.summary()
    return model

### function for creating a model for tuning other hyper parameter

In [ ]:
def create_model_2(N_hidden_nodes, l_rate, Batch_size, epochs):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    model = keras.Sequential([
        keras.layers.Dense(N_hidden_nodes, activation=tf.nn.leaky_relu, input_shape=(4,)),
        keras.layers.Dense(21,activation='linear')
    ])
    optimizer=tf.keras.optimizers.Adamax(learning_rate=l_rate, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.0)
    model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse'])
    model.summary()
    model.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=epochs, 
                        verbose=0, validation_split=0.1)
    return model


### function for creating a model for all tuned hyperparameter(actual model)

In [ ]:
def build_model(N_hidden_nodes, input_dim, N_outputs, l_rate, Batch_size, Epochs):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    callbacks = [
        EarlyStopping(monitor='val_mse', patience=100, verbose=2),
        ModelCheckpoint('best_model.h5', monitor='val_mse', save_best_only=True, verbose=0)
    ]
    model = keras.Sequential([
        keras.layers.Dense(N_hidden_nodes, activation=tf.nn.leaky_relu, input_shape=(input_dim,)),
    # keras.layers.Dropout(0.5),
        keras.layers.Dense(N_outputs,activation='linear')
    ])
    model.compile(loss='mse',
                optimizer=tf.keras.optimizers.Adamax(learning_rate=l_rate, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.0),
                metrics=['mse'])
    model.summary()
    history = model.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=Epochs, 
                        verbose=1, validation_split=0.1, callbacks=callbacks)
    return model, history

### Uniform Spaced Sampling(only for this dataset of weld residue stress, if we got other data, this might need to be deleted or modified)

In [ ]:
def uniformly_spaced_sampling(y_label, y_label_new, y):
    '''
    fit and interpolate
    '''
    f = interpolate.interp1d(y_label, y, kind='linear') # linear interpolation function use first row of y and all of y -> linear interpolate location and y value
    ynew=pd.DataFrame(f(y_label_new)) #first row of y (linear distributed) -> linearly interpolate(predict) when in evenly distributed location, what r y
    return ynew

### assign column names to data set 
can see only one column of y here

In [ ]:
def data_melt(x, y):
    '''
    Add position as input
    input:
        x: old input 
        y: old output
    output:
        x_new: new input 
        y_new: new output
    '''
    y_label_str = [str(x) for x in y_label_new] # make a list of string of y_label_new
    dataset = pd.concat([x, y],axis=1, ignore_index=True)
    col_names = xcolumn + y_label_str
    dataset.columns = col_names
    dataset = dataset.melt(id_vars=xcolumn, 
        var_name=yparameter, 
        value_name=yname) #for given welding parameter, in a given location, the stress is in this dataset
    x_new = dataset.iloc[:, 0:(len(xcolumn)+1)]
    y_new = dataset.iloc[:, (len(xcolumn)+1)]
    return x_new, y_new

### define mean square error function MSE

In [ ]:
def MSE(testY, predicY):
    '''
    Get MSE fun
    '''
    MSE=np.sum(np.power((testY - predicY),2))/testY.shape[1]/testY.shape[0]
    return MSE

### Define a function getting number of output

In [ ]:
def get_N_output(Y_train):
    '''
    Get the number of outputs
    '''
    if Y_train.ndim == 1:
        N_outputs = 1
    else:
        N_outputs = Y_train.shape[1]
    return N_outputs

### Define a function getting y(output) label
use y in the csv
the first row of that csv is y_lab
y_lab_new is min to max of y_lab in length of y_lab

In [ ]:
def get_y_lable(root):
    '''
    Create an array for uniform interval depth 
    (Used biased mesh in simulaiton, so the sample point depth is not uniform) 
    input:
        root: file root contains the sample point depth information
    outputs:
        y_lab: old sample point position array
        y_lab_new: new sample point position array
    '''
    x, y = data_import(root)
    y_lab = y.iloc[0, :]
    y_lab_new = np.linspace(round(min(y_lab),2), 
                              round(max(y_lab),2), 
                              round(len(y_lab),2))
    return y_lab, y_lab_new
    

### Prepocessing function

In [ ]:
def pre_processing(model_type, x, y):
    '''
    Data preprocessing (Uniformly spaced sampling, normalisation, train test split)
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)        
        x: a dataframe of inputs of the whole dataset
        y: a dataframe of outputs of the whole dataset
    outputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        Y_test: Test output
    '''
      # uniformly_spaced_sampling
    global y_label, y_label_new
    y_label, y_label_new = get_y_lable(r'C:\Users\wingt\Downloads\benchmark_BD1.csv')
    y = uniformly_spaced_sampling(y_label, y_label_new, y)

    # Split dataset
    X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=1/125.0, random_state=3)
    X_train=pd.DataFrame(X_train[:].values)
    X_test =pd.DataFrame(X_test[:].values)
    Y_train=pd.DataFrame(Y_train[:].values)
    Y_test=pd.DataFrame(Y_test[:].values)
    Y_train = uniformly_spaced_sampling(y_label, y_label_new, Y_train)
    Y_test = uniformly_spaced_sampling(y_label, y_label_new, Y_test)
    # data reconstruction
    if 'ANN2' in model_type:
        X_train, Y_train = data_melt(X_train, Y_train)
        X_test, Y_test_1 = data_melt(X_test, Y_test)

    # Normalization
    global scaler_X, scaler_Y
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    scaled_train_X = scaler_X.fit_transform(X_train.to_numpy())
    scaled_test_X = scaler_X.transform(X_test)

    if Y_train.ndim == 1:
      Y_train = np.array(Y_train).reshape(-1,1)#column array
    scaled_train_Y = scaler_Y.fit_transform(Y_train)
    Proc_X_train = scaled_train_X
    Proc_Y_train = scaled_train_Y
    Proc_X_test = scaled_test_X

    return model_type, Proc_X_train, Proc_Y_train, Proc_X_test, Y_test

### Get Result function
build a model based on preprocessed data and tuned hyperparameter
output the model, predicted scalar transformed y, predicted y in 21 columns, history of fitting

In [ ]:
def get_result(model_type, Proc_X_train, Proc_Y_train, Proc_X_test, N_neurons, Batch_size, Epochs, l_rate):
    
    '''
    Get the trained model, prediction results, and NN training history
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        N_neurons: Number of Neurons in the hidden layer
        l_rate: Learning rate
    outputs:
        model: Trained model
        predict_test: predicted results
        history: training history
    '''
    #build model
    N_inputs = Proc_X_train.shape[1]
    N_outputs = get_N_output(Proc_Y_train)
    model, history = build_model(N_neurons, N_inputs, N_outputs, l_rate, Batch_size, Epochs)
    # Predict
    
    predict_test_scal = model.predict(Proc_X_test)
    if 'ANN2' in model_type:
      predict_test_scal = np.reshape(predict_test_scal, (21, -1)).T
    
    predict_test = scaler_Y.inverse_transform(predict_test_scal)
    return  model, predict_test_scal, predict_test, history

### fix random seed

In [ ]:
def seed_tensorflow(seed):
    '''
    Fix ramdom seed
    '''
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED'] = str(seed)

### actual import and train test split


In [ ]:
# Fix ramdom seed
warnings.filterwarnings('ignore')
seed_tensorflow(42)

# Import dataset
x, y = data_import(r'C:\Users\wingt\Downloads\S11_along_BD_205.csv')

# Pre-processing
model_type, Proc_X_train, Proc_Y_train, Proc_X_test, Y_test = pre_processing('ANN2', x, y)

### function of tuning(timed)

In [ ]:
def tuning(tuning=False): # Run Bayesian Optimization: find maximum value of unknown function by a few iteration
    if tuning==True:
        import time
        t1=time.time
        tune_optimizer()
        t2=time.time
        nn_bo = BayesianOptimization(nn_cl_bo, BO_turing_other_params_nn, random_state=5)
        nn_bo.maximize(init_points=no_iter_before_conclude, n_iter=no_iter_conclude)
        params_nn_ = nn_bo.max['params']
        print(params_nn_)
        print(nn_bo)
        t3=time.time
        tune_other_para()
        t4=time.time
        print('required Optimizer tuning time:',t2-t1)
        print('required BO time:',t3-t2)
        print('required Gridsearch others time:',t4-t3)

### The actual implementation

In [22]:
# Build model and Predict
tuning()
ANN, predict_test_scal,predict_test, history = get_result(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,
                                      1000, 8, 1000, 0.001)
Y_test_scal = scaler_Y.transform(Y_test.values.reshape(-1,1)).reshape(-1, 21)
MSE_R_scal = MSE(Y_test_scal, predict_test_scal)
MSE_R = MSE(Y_test.values, predict_test)
print(MSE_R_scal)
print(MSE_R)
plot_history(history)
# save the model
# ANN.save(r'data_analysis/saved_model/ANN.h5')
# pickle.dump(scaler_X, open('scaler_X.pkl','wb'))
# pickle.dump(scaler_Y, open('scaler_Y.pkl','wb'))

#for i in range(len(predict_test)):
get_plot(Y_test.iloc[i, :], predict_test[i], i)

pred_results = []
sim_results = []
for i in range(len(predict_test)):
    for j in range(len(predict_test[0])):
        sim_results.append(Y_test.iloc[i, j])
        pred_results.append(predict_test[i][j])

max_results = max(max(pred_results,sim_results))
min_results = min(min(pred_results,sim_results))
fig,ax = plt.subplots(figsize=(12,12))
ax.plot([min_results, max_results], [min_results, max_results],'b', linewidth=3)
ax.scatter(pred_results, sim_results,s=10, c='r')
ax.set_xlabel("Surrogate model predicted residual stress (MPa)", fontsize=24)
ax.set_ylabel("Finite element simulated residual stress (MPa)", fontsize=24)
plt.legend()
plt.show()

IndentationError: unexpected indent (2823809950.py, line 17)

In [25]:
from keras.layers import LeakyReLU
act_tun_number_sample_used_in_one_training=10
act_tun_number_of_full_update=200
activation_function=['relu',LeakyReLU(alpha=0.01),'hard_sigmoid','elu','linear','sigmoid','softmax']
param_grid = dict(activation_function=activation_function)
def tune_act_funct():
    # callbacks = [
    #     EarlyStopping(monitor='mse', patience=100, verbose=0),
    # ]
    model = KerasRegressor(build_fn=create_model_for_tune_act, 
                           nb_epoch=act_tun_number_of_full_update, 
                           batch_size=act_tun_number_sample_used_in_one_training, 
                           verbose=1) 
    activation = activation_function_list # 200 hidden nodes
    param_grid = dict(activation=activation) 
    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=8)
    grid_result = grid.fit(Proc_X_train, Proc_Y_train)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 
    for index, value in enumerate(grid_result.cv_results_['mean_test_score']):
        print("%f (%f) with: %r" % (grid_result.cv_results_['mean_test_score'][index], 
                                    grid_result.cv_results_['std_test_score'][index], 
                                    grid_result.cv_results_['params'][index]))
def create_model_for_tune_act(activation='relu'):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    model = keras.Sequential([
        keras.layers.Dense(200, activation=activation, input_shape=(4,)),
        keras.layers.Dense(21,activation='linear')
    ])
    model.compile(loss='mse',
                optimizer='adam',
                metrics=['mse'])
    model.summary()
    return model        
tune_act_funct()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               1000      
_________________________________________________________________
dense_1 (Dense)              (None, 21)                4221      
Total params: 5,221
Trainable params: 5,221
Non-trainable params: 0
_________________________________________________________________
427/427 [==============================] - 0s 664us/step - loss: 0.1822 - mse: 0.1822
Best: -0.162955 using {'activation': 'relu'}
-0.162955 (0.099280) with: {'activation': 'relu'}
-0.191986 (0.142865) with: {'activation': <tensorflow.python.keras.layers.advanced_activations.LeakyReLU object at 0x000001B49BFA2850>}
-0.372672 (0.322990) with: {'activation': 'hard_sigmoid'}
-0.481071 (0.316380) with: {'activation': 'elu'}
-0.483844 (0.313194) with: {'activation': 'linear'}
-0.414365 (0.295534) with: {'activation'